<a href="https://colab.research.google.com/github/jhyeon-kim/ai_study/blob/main/Chapter1_2_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

In [1]:
!pip uninstall cudf-cu12 ibis-framework bigframes
!pip install datasets

Found existing installation: cudf-cu12 24.4.1
Uninstalling cudf-cu12-24.4.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/cudf/*
    /usr/local/lib/python3.10/dist-packages/cudf_cu12-24.4.1.dist-info/*
Proceed (Y/n)? Y
  Successfully uninstalled cudf-cu12-24.4.1
Found existing installation: ibis-framework 9.2.0
Uninstalling ibis-framework-9.2.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/ibis/*
    /usr/local/lib/python3.10/dist-packages/ibis_framework-9.2.0.dist-info/*
Proceed (Y/n)? Y
  Successfully uninstalled ibis-framework-9.2.0
Found existing installation: bigframes 1.18.0
Uninstalling bigframes-1.18.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/bigframes-1.18.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/bigframes/*
    /usr/local/lib/python3.10/dist-packages/bigframes_vendored/*
Proceed (Y/n)? Y
  Successfully uninstalled bigframes-1.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 4.1 MB/s eta 0:00:00


In [2]:
!pip install sacremoses
# 아래 코드 실행 시 RuntimeError: Missing dependencies: sacremoses 에러가 나서 추가

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.4 MB/s eta 0:00:00


In [3]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [4]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model) # 여기까지는 Q 와 K 벡터 사이 내적을 통한 각 토큰끼리의 유관한 정도 (유사도)

    if mask is not None:
      score = score + (mask * -1e9)

    score = self.softmax(score) # mask 에 의해서 아주 작은(아주 절댓값 큰 음수일 확률 높음)가 된 경우에, softmax 를 타면 e의 지수로 score 가 올라가면서 거의 0에 가깝게 됨.
    result = torch.matmul(score, v) # 각 토큰에 대한 벡터(v) 에 대해, 다른 토큰과의 유사도를 기반으로 가중합해서 조정함.
    result = self.dense(result)

    return result

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [5]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [6]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model)) # 차원(d_model)이 커질수록 값이 천천히 변하도록 설정됨
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [7]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, 1) # 한 차원만으로 표현 가능한 이진 분류(긍/부정)이므로

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model) # 임베딩 벡터에 대한 스케일링
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = TextClassifier(len(tokenizer), 32, 2, 32) # 어휘크기, d_model 크기, 트랜스포머 레이어 수, FFN 의 차원 수

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [8]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [9]:
import numpy as np
import matplotlib.pyplot as plt


def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0] # 예측값을 이진 분류로 변환하기 위해 T/F 로 나누고 -> 정수로 변환 -> 첫 번째 차원만 사용

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [10]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 231.39130347967148
=========> Train acc: 0.770 | Test acc: 0.739
Epoch   1 | Train Loss: 175.87362387776375
=========> Train acc: 0.834 | Test acc: 0.782
Epoch   2 | Train Loss: 150.798003166914
=========> Train acc: 0.866 | Test acc: 0.794
Epoch   3 | Train Loss: 131.16758073866367
=========> Train acc: 0.888 | Test acc: 0.806
Epoch   4 | Train Loss: 112.27360501885414
=========> Train acc: 0.914 | Test acc: 0.804
Epoch   5 | Train Loss: 94.90008299052715
=========> Train acc: 0.935 | Test acc: 0.806
Epoch   6 | Train Loss: 78.9693440683186
=========> Train acc: 0.949 | Test acc: 0.808
Epoch   7 | Train Loss: 65.03845735266805
=========> Train acc: 0.959 | Test acc: 0.806
Epoch   8 | Train Loss: 52.379075014963746
=========> Train acc: 0.961 | Test acc: 0.793
Epoch   9 | Train Loss: 43.1345933964476
=========> Train acc: 0.962 | Test acc: 0.798
Epoch  10 | Train Loss: 34.51853860449046
=========> Train acc: 0.984 | Test acc: 0.795
Epoch  11 | Train Loss: 26.642

학습이 안정적으로 진행되며 RNN보다 빨리 수렴하는 것을 확인할 수 있습니다.
하지만 test 정확도가 RNN보다 낮은 것을 보았을 때, overfitting에 취약하다는 것을 알 수 있습니다.

- [ ]  Multi-head attention(MHA) 구현
    - Self-attention module을 MHA로 확장해주시면 됩니다. 여기서 MHA는 다음과 같이 구현합니다.
        1. 기존의 $W_q, W_k, W_v$를 사용하여 $Q, K, V$를 생성합니다. 이 부분은 코드 수정이 필요 없습니다.
            1. $Q, K, V \in \mathbb{R}^{S \times D}$가 있을 때, 이를 $Q, K, V \in \mathbb{R}^{S \times H \times D’}$으로 reshape 해줍니다. 여기서 $H$는 `n_heads`라는 인자로 받아야 하고, $D$가 $H$로 나눠 떨어지는 값이여야 하는 제약 조건이 필요합니다. $D = H \times D’$입니다.
            2. $Q, K, V$를 $Q, K, V \in \mathbb{R}^{H \times S \times D’}$의 shape으로 transpose해줍니다.
        2. $A = QK^T/\sqrt{D'} \in \mathbb{R}^{H \times S \times S}$를 기존의 self-attention과 똑같이 계산합니다. 이 부분은 코드 수정이 필요 없습니다.
        3. Mask를 더합니다. 기존과 $A$의 shape이 달라졌기 때문에 dimension을 어떻게 맞춰줘야할지 생각해줘야 합니다.
        4. $\hat{x} = \textrm{Softmax}(A)V \in \mathbb{R}^{H \times S \times D'}$를 계산해주고 transpose와 reshape을 통해 $\hat{x} \in \mathbb{R}^{S \times D}$의 shape으로 다시 만들어줍니다.
        5. 기존과 똑같이 $\hat{x} = \hat{x} W_o$를 곱해줘서 마무리 해줍니다. 이 또한 코드 수정이 필요 없습니다.

In [11]:
import torch
from torch import nn
from math import sqrt

class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, d_model, n_heads):
        super().__init__()

        assert d_model % n_heads == 0, f"차원 수는 n_heads 로 나누어 떨어져야 합니다!"

        self.input_dim = input_dim
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads  # D'(d_head) = D / H (하나의 헤드가 처리하게 되는 차원의 크기)

        # Q, K, V의 생성은 기존 Self-Attention과 동일
        self.wq = nn.Linear(input_dim, d_model)
        self.wk = nn.Linear(input_dim, d_model)
        self.wv = nn.Linear(input_dim, d_model)
        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def split_heads(self, x, batch_size):
        """
        벡터를 여러 헤드로 나누고 차원 맞춤
        x: (batch_size, seq_len, d_model) -> (batch_size, n_heads, seq_len, d_head)
        """
        x = x.view(batch_size, -1, self.n_heads, self.d_head)
        return x.transpose(1, 2)  # (batch_size, n_heads, seq_len, d_head)

    def forward(self, x, mask):
        batch_size = x.shape[0]

        # Q, K, V 계산
        q = self.wq(x)  # (batch_size, seq_len, d_model)
        k = self.wk(x)  # (batch_size, seq_len, d_model)
        v = self.wv(x)  # (batch_size, seq_len, d_model)

        # 헤드로 나누기 (split heads)
        q = self.split_heads(q, batch_size)  # (batch_size, n_heads, seq_len, d_head)
        k = self.split_heads(k, batch_size)  # (batch_size, n_heads, seq_len, d_head)
        v = self.split_heads(v, batch_size)  # (batch_size, n_heads, seq_len, d_head)

        # QK^T 계산 (어텐션 스코어)
        score = torch.matmul(q, k.transpose(-1, -2))  # K 에 대해 마지막 두 차원(depth와 seq_len)을 바꾸는 역할을 해서 QK^T의 내적을 가능하게 하기
        score = score / sqrt(self.d_head)  # D'로 나눔


        # 마스크 적용 (mask의 차원 맞추기)
        if mask is not None:
            mask = mask.unsqueeze(1)  # (batch_size, 1, 1, seq_len) 이래도 되는 이유는 pytorch 의 broadcasting
            score = score + (mask * -1e9)

        # 소프트맥스
        score = self.softmax(score)  # (batch_size, n_heads, seq_len, seq_len)

        # 가중합 (softmax(score) * V)
        result = torch.matmul(score, v)  # (batch_size, n_heads, seq_len, d_head)

        # 헤드를 다시 합치기 (reshape to (batch_size, seq_len, d_model))
        result = result.transpose(1, 2).contiguous()  # (batch_size, seq_len, n_heads, d_head)
        result = result.view(batch_size, -1, self.d_model)  # (batch_size, seq_len, d_model)

        # 최종 선형 변환 (W_o 곱하기)
        result = self.dense(result)

        return result


- [ ]  Layer normalization, dropout, residual connection 구현
    - 다시 `TransformerLayer` class로 돌아와서 과제를 진행하시면 됩니다.
    - Attention module을 $MHA$, feed-forward layer를 $FFN$이라고 하겠습니다.
    - 기존의 구현은 다음과 같습니다:
        
        ```python
        # x, mask is given
        
        x1 = MHA(x, mask)
        x2 = FFN(x1)
        
        return x2
        ```
        
    - 다음과 같이 수정해주시면 됩니다.
        
        ```python
        # x, mask is given
        
        x1 = MHA(x, mask)
        x1 = Dropout(x1)
        x1 = LayerNormalization(x1 + x)
        
        x2 = FFN(x)
        x2 = Dropout(x2)
        x2 = LayerNormalization(x2 + x1)
        
        return x2
        ```
        
    - 여기서 `x1 + x`와 `x2 + x1`에 해당하는 부분들은 residual connection이라고 부릅니다.

In [12]:
import torch
from torch import nn


class TransformerLayerWithMHA(nn.Module):
    def __init__(self, input_dim, d_model, dff, n_heads, dropout_rate=0.1):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model
        self.dff = dff
        self.n_heads = n_heads

        # Multi-Head Attention (MHA) 설정
        self.mha = MultiHeadAttention(input_dim, d_model, n_heads)

        # Feed-forward 네트워크 설정 (FFN)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Linear(dff, d_model)
        )

        # Dropout 및 Layer Normalization 설정
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)

    def forward(self, x, mask):
        # Multi-Head Attention (MHA)
        x1 = self.mha(x, mask)  # MHA
        x1 = self.dropout1(x1)  # Dropout after MHA
        x1 = self.layernorm1(x1 + x)  # Residual connection and Layer Norm (x1 + x)

        # Feed-Forward Network (FFN)
        x2 = self.ffn(x1)  # FFN
        x2 = self.dropout2(x2)  # Dropout after FFN
        x2 = self.layernorm2(x2 + x1)  # Residual connection and Layer Norm (x2 + x1)

        return x2



- [ ]  5-layer 4-head Transformer
- 기존 실습에서 사용한 hyper-parameter들과 위에서 구현한 Transformer를 가지고 5-layer 4-head Transformer의 성능 결과를 report해주시면 됩니다.

In [13]:

# 5-layer, 4-head Transformer 설정
class NewTextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff, n_heads, dropout_rate=0.1):
    super().__init__()

    self.vocab_size = vocab_size
    self.d_model = d_model
    self.n_layers = n_layers
    self.dff = dff

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    self.layers = nn.ModuleList([TransformerLayerWithMHA(d_model, d_model, dff, n_heads) for _ in range(n_layers)])
    self.classification = nn.Linear(d_model, 1) # 한 차원만으로 표현 가능한 이진 분류(긍/부정)이므로

  def forward(self, x):
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]
    seq_len = x.shape[1]

    x = self.embedding(x)
    x = x * sqrt(self.d_model) # 임베딩 벡터에 대한 스케일링
    x = x + self.pos_encoding[:, :seq_len]

    for layer in self.layers:
      x = layer(x, mask)

    x = x[:, 0]
    x = self.classification(x)

    return x


model = NewTextClassifier(len(tokenizer), 32, 5, 32, 4) # 어휘크기, d_model 크기, 트랜스포머 레이어 수, FFN 의 차원 수, 헤드 수
model = model.to('cuda')

In [14]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.BCEWithLogitsLoss()
n_epochs = 50

optimizer = Adam(model.parameters(), lr=lr)
for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 214.37349662184715
=========> Train acc: 0.836 | Test acc: 0.803
Epoch   1 | Train Loss: 143.63034015893936
=========> Train acc: 0.877 | Test acc: 0.824
Epoch   2 | Train Loss: 114.9144117385149
=========> Train acc: 0.917 | Test acc: 0.835
Epoch   3 | Train Loss: 91.14024514332414
=========> Train acc: 0.937 | Test acc: 0.836
Epoch   4 | Train Loss: 70.70465908199549
=========> Train acc: 0.964 | Test acc: 0.838
Epoch   5 | Train Loss: 56.1495560798794
=========> Train acc: 0.967 | Test acc: 0.829
Epoch   6 | Train Loss: 41.96754351537675
=========> Train acc: 0.983 | Test acc: 0.834
Epoch   7 | Train Loss: 33.20370359905064
=========> Train acc: 0.985 | Test acc: 0.827
Epoch   8 | Train Loss: 26.223944963421673
=========> Train acc: 0.987 | Test acc: 0.828
Epoch   9 | Train Loss: 22.793878309428692
=========> Train acc: 0.992 | Test acc: 0.831
Epoch  10 | Train Loss: 18.705248292302713
=========> Train acc: 0.984 | Test acc: 0.821
Epoch  11 | Train Loss: 19.3